In [ ]:
from visio_gptq.model import GPTQModel
model = GPTQModel(model_name="anon8231489123/vicuna-13b-GPTQ-4bit-128g", device="cuda", wbits=4, groupsize=128)

In [ ]:
from langchain.llms.base import LLM
from typing import Optional, List

class GPTQModelLLM(LLM):
    model: GPTQModel
    params: dict = {}

    @property
    def _llm_type(self) -> str:
        return "gptq_model_llm"

    def callbackFunc(self,txt):
        print(txt, end=" ", flush=True)

    def _call(self, prompt:str, stop: Optional[List[str]] = None) -> str:
        current_stop = self.params.get("stop", [])
        if stop is not None:
            current_stop.extend(stop)
            self.params["stop"] = current_stop

        conv = model.inference(prompt=prompt, **self.params)
        if isinstance(conv, str):
            return conv
        return "Error"

In [ ]:
from langchain.chains import ConversationChain
from langchain.prompts import PromptTemplate
llm = GPTQModelLLM(model=model, params={"temperature": 0.0, "stop":["Question:", "###"]})
DEFAULT_PROMPT = PromptTemplate(
    input_variables=["history","input"],
    template="The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know."
            "Current conversation:"
            "{history}"
            "### Human: {input}"
            "### AI:")
conv = ConversationChain(llm=llm, prompt=DEFAULT_PROMPT, verbose=False)

In [ ]:
result = conv.predict(input="What is the capital of Portugal?")
print(result)

In [ ]:
result = conv.predict(input="and Spain?")
print(result)

In [ ]:
result = conv.predict(input="What was my first question?")
print(result)

In [ ]:
from langchain.agents import load_tools, initialize_agent, AgentType
import os

os.environ["SERPAPI_API_KEY"] = "YOUR API KEY HERE"

tools_names = ["serpapi"]
tools = load_tools(tools_names)

In [ ]:
agent = initialize_agent(llm=llm, tools=tools, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True)

In [ ]:
agent.run("What is LangChain?")

In [ ]:
agent.run("who is the ceo of pipe?")

In [ ]:
agent.run("What is the cheapest house in portugal?")